<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/A3_adapter_seq_bn_classification_C_5_P_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This tests using an adapter for pretraining





# Setup

In [1]:
# Pretraining Dataset Settings
pretrain_dataset_name = "BigTMiami/amazon_split_25M_reviews_20_percent_condensed"
# 5% of 20% creates a 1% - I think this should take 15 minutes
pretrain_percentage = "5"
pretrain_train_split = f"train[:{pretrain_percentage}%]"
pretrain_eval_split = "dev"

# Pretraining Adapter settings
experiment_tag = "A3"
adapter_short_desc = "classification_from_pretraining"
adapter_type = "seq_bn" # could be "lora", etc.

pretrained_adapter_hub_name = f"{experiment_tag}_adapter_{adapter_type}_{adapter_short_desc}_P_{pretrain_percentage}"
pretrained_adapter_name = f"{experiment_tag}_adapter_{adapter_type}_{adapter_short_desc}_P_{pretrain_percentage}"
print(f"Pretrained Adapter Hub Name: {pretrained_adapter_hub_name}")

# No Pretraining Classification Settings
classification_dataset_name = "BigTMiami/amazon_helpfulness"
classification_train_percentage = "5"
classification_train_split = f"train[:{classification_train_percentage}%]"
classification_eval_split = "dev"

classification_adapter_hub_name = f"{experiment_tag}_adapter_{adapter_type}_{adapter_short_desc}_C_{classification_train_percentage}"
classification_adapter_name = f"{experiment_tag}_adapter_{adapter_type}_{adapter_short_desc}_C_{classification_train_percentage}"

print(f"Classifer Not Pretrained Adapter Hub Name: {classification_adapter_hub_name}")

# Pretrained Classification Settings


# Classification using Pretrained Settings
# REPLACE WITH ABOVE WHEN ALL SET
pretrained_adapter_hub_name = "BigTMiami/A1_adapter_seq_bn_pretrain_P_5"
classification_adapter_from_pretrained_hub_name = f"{experiment_tag}_adapter_{adapter_type}_{adapter_short_desc}_P_{pretrain_percentage}_C_{classification_train_percentage}"


Pretrained Adapter Hub Name: A3_adapter_seq_bn_classification_from_pretraining_P_5
Classifer Not Pretrained Adapter Hub Name: A3_adapter_seq_bn_classification_from_pretraining_C_5


In [ ]:
# from google.colab import drive
# drive.mount("/content/drive")
# %cd '/content/drive/MyDrive/AdaptOrDie'

In [2]:
!pip install datasets
!pip install transformers[torch]
!pip install adapters
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 5.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86

In [3]:
!pip install pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.7 MB/s eta 0:00:00


In [4]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    gpu_used = info.used//1024**2
    print(f"GPU {gpu_used} MB")

print_gpu_utilization()

GPU 233 MB


In [5]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

In [6]:
from datasets import load_dataset

classification_train_dataset, classification_eval_dataset= load_dataset(classification_dataset_name, split=[classification_train_split,classification_eval_split])

print(classification_train_dataset)
print(classification_eval_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 5763
})
Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 5000
})


In [ ]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
classficiation_no_pretrain_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

In [ ]:
# adapter_hub_name = "adapter_classifier_from_prtrained_test"
# adapter_name = model.load_adapter("BigTMiami/domain_adapter_test")
# # Add head for masked language modeling
# model.add_classification_head(
#     adapter_name,
#     num_labels=2,
#     id2label={ 0: "unhelpful", 1: "helpful"}, overwrite_ok=True
#   )

# # Set the adapter to be used for training
# model.train_adapter(adapter_name)

In [ ]:
# Add a new adapter
classficiation_no_pretrain_model.add_adapter(classification_adapter_name, config=adapter_type)

# Add head for classification modeling
classficiation_no_pretrain_model.add_classification_head(
    classification_adapter_name,
    num_labels=2,
    id2label={ 0: "unhelpful", 1: "helpful"})

# Set the adapter to be used for training
classficiation_no_pretrain_model.train_adapter(classification_adapter_name)

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
print_gpu_utilization()

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

GPU 233 MB


In [8]:
from transformers import DataCollatorWithPadding

classification_data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Classification with No Pretraining

In [ ]:
from transformers import TrainingArguments

classification_training_args = TrainingArguments(
    output_dir=classification_adapter_hub_name,
    learning_rate=1e-4,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=250,
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

In [ ]:
from adapters import AdapterTrainer

classification_trainer = AdapterTrainer(
    model=classficiation_no_pretrain_model,
    args=classification_training_args,
    train_dataset=classification_train_dataset,
    eval_dataset=classification_eval_dataset,
    data_collator=classification_data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
classification_training_results = classification_trainer.train()
print(classification_training_results)

In [ ]:
classification_eval_results = classification_trainer.evaluate()
classification_eval_loss = classification_eval_results["eval_loss"]
classification_eval_f1 = classification_eval_results["eval_f1_macro"]
print(f"Classification no Pretraining Eval F1:{100.0 * classification_eval_f1:.2f}")
print(f"Classification no Pretraining Eval Loss: {classification_eval_loss:.2f}")
print(classification_eval_results)

In [ ]:
classficiation_no_pretrain_model.push_adapter_to_hub(
    classification_adapter_hub_name,
    classification_adapter_name,
    datasets_tag=classification_dataset_name
)
print(f"Pushed {classification_adapter_hub_name}")

In [ ]:
import gc

print_gpu_utilization()

del classficiation_no_pretrain_model
del classification_trainer
gc.collect()

print_gpu_utilization()

# Classification from Pre-training

In [9]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
classficiation_from_pretrain_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
pretrained_adapter_hub_name = "BigTMiami/A1_adapter_seq_bn_pretrain_P_5"

# Load Pretrained adapter without head
loaded_adapter_name = classficiation_from_pretrain_model.load_adapter(pretrained_adapter_hub_name, with_head=False)

# Add head for classification modeling
classficiation_from_pretrain_model.add_classification_head(
    loaded_adapter_name,
    num_labels=2,
    id2label={ 0: "unhelpful", 1: "helpful"})

# Set the adapter to be used for training
classficiation_from_pretrain_model.train_adapter(loaded_adapter_name)

(…)b/master/dist/v2/index/roberta-base.json:   0%|          | 0.00/540 [00:00<?, ?B/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

head_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

pytorch_model_head.bin:   0%|          | 0.00/157M [00:00<?, ?B/s]

In [11]:
from transformers import TrainingArguments

classification_from_pretrained_training_args = TrainingArguments(
    output_dir=classification_adapter_from_pretrained_hub_name,
    learning_rate=1e-4,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=250,
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

In [12]:
from adapters import AdapterTrainer

classification_from_pretrained_trainer = AdapterTrainer(
    model=classficiation_from_pretrain_model,
    args=classification_from_pretrained_training_args,
    train_dataset=classification_train_dataset,
    eval_dataset=classification_eval_dataset,
    data_collator=classification_data_collator,
    compute_metrics=compute_metrics,
)
print_gpu_utilization()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


GPU 1075 MB


In [13]:
classification_from_pretrained_training_results = classification_from_pretrained_trainer.train()
print(classification_from_pretrained_training_results)
print_gpu_utilization()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
250,0.401500
500,0.358800
750,0.333300
1000,0.315000


TrainOutput(global_step=1083, training_loss=0.3522756282422461, metrics={'train_runtime': 348.9615, 'train_samples_per_second': 49.544, 'train_steps_per_second': 3.103, 'total_flos': 4359775986565506.0, 'train_loss': 0.3522756282422461, 'epoch': 3.0})
GPU 9629 MB


In [14]:
classification_from_pretrained_eval_results = classification_from_pretrained_trainer.evaluate()
classification_from_pretrained_eval_loss = classification_from_pretrained_eval_results["eval_loss"]
classification_from_pretrained_eval_f1 = classification_from_pretrained_eval_results["eval_f1_macro"]
print(f"Classification with Pretraining Eval F1:{100.0 * classification_from_pretrained_eval_loss:.2f}")
print(f"Classification with Pretraining Eval Loss: {classification_from_pretrained_eval_f1:.2f}")
print(classification_from_pretrained_eval_results)
print_gpu_utilization()

Classification with Pretraining Eval F1:34.62
Classification with Pretraining Eval Loss: 0.52
{'eval_loss': 0.34615397453308105, 'eval_accuracy': 0.8574, 'eval_f1_macro': 0.5150714830882369, 'eval_runtime': 47.513, 'eval_samples_per_second': 105.234, 'eval_steps_per_second': 6.588, 'epoch': 3.0}
GPU 9629 MB


In [15]:
classficiation_from_pretrain_model.push_adapter_to_hub(
    classification_adapter_from_pretrained_hub_name,
    loaded_adapter_name,
    datasets_tag=classification_dataset_name
)
print(f"Pushed {classification_adapter_from_pretrained_hub_name}")

pytorch_model_head.bin:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

Pushed A3_adapter_seq_bn_classification_from_pretraining_P_5_C_5


In [16]:
import gc

print_gpu_utilization()

del classficiation_from_pretrain_model
del classification_from_pretrained_trainer
gc.collect()

print_gpu_utilization()

GPU 9629 MB
GPU 9629 MB


In [17]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()

Disconnecting Session
